# Imports

In [ ]:
import pandas as pd
from glob import glob
from tqdm import tqdm

# Load data

In [ ]:
files_1 = glob('/scratch/tsoares/wikidumps/simplewiki-NS0-20230901/processed_data/links*')
files_2 = glob('/scratch/tsoares/wikidumps/simplewiki-NS0-20231001/processed_data/links*')
print(files_1)
print(files_2)

In [ ]:
dfs = []
for file in files_1:
    dfs.append(pd.read_parquet(file))
df_1 = pd.concat(dfs).reset_index(drop=True)
df_1

In [ ]:
dfs = []
for file in files_2:
    dfs.append(pd.read_parquet(file))
df_2 = pd.concat(dfs).reset_index(drop=True)
df_2

# Convert Data Into Better Structure

In [ ]:
df_1 = df_1.to_dict('records')
df_2 = df_2.to_dict('records')

In [ ]:
early_data = {}
for link in tqdm(df_1):
    if link['source_title'] not in early_data:
        early_data[link['source_title']] = {}
    if link['target_title'] not in early_data[link['source_title']]:
        early_data[link['source_title']][link['target_title']] = []
    early_data[link['source_title']][link['target_title']].append(link)
early_data

In [ ]:
later_data = {}
for link in tqdm(df_2):
    if link['source_title'] not in later_data:
        later_data[link['source_title']] = {}
    if link['target_title'] not in later_data[link['source_title']]:
        later_data[link['source_title']][link['target_title']] = []
    later_data[link['source_title']][link['target_title']].append(link)

# Compare later data with earlier data

In [ ]:
new_pages = 0
new_page_links = 0
new_links = []
sure_new_links = 0

for source_page in tqdm(later_data):
    if source_page not in early_data:
        new_pages += 1
        new_page_links += len(later_data[source_page])
        continue
    for target_page in later_data[source_page]:
        if target_page not in early_data[source_page]:
            new_links.extend(later_data[source_page][target_page])
        else:
            links_with_id = []
            links_without_id = []
            for link in later_data[source_page][target_page]:
                if link['link_ID'] is None:
                    links_without_id.append(link)
                else:
                    links_with_id.append(link)
            for link in links_with_id:
                found = False
                for old_link in early_data[source_page][target_page]:
                    if link['link_ID'] == old_link['link_ID']:
                        found = True
                        break
                if not found:
                    new_links.append(link)
                    sure_new_links += 1
            
            used = set([])
            for new_link in links_without_id:
                for i, old_link in enumerate(early_data[source_page][target_page]):
                    if old_link['link_ID'] is None and old_link['mention'] == new_link['mention'] and i not in used:
                        used.add(i)
                        break
                    if i == len(early_data[source_page][target_page]) - 1:
                        new_links.append(new_link)
                        sure_new_links += 1
                            
print(f"The new data has {new_pages} new pages containing {new_page_links} links")
print(f"There are potentially {len(new_links)} new links, out of which {sure_new_links} are sure new links")        

In [ ]:
link_struc = {}
for link in new_links:
    if link['source_ID'] in link_struc:
        link_struc[link['source_ID']].append(link['mention'])
    else:
        link_struc[link['source_ID']] = [link['mention']]
link_struc